### Import Modules

In [42]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time
from importlib import reload

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [63]:
reload(dl)
reload(sf)
reload(fit)
reload(bs)

<module 'fitter.bootstrapper' from '../fitter/bootstrapper.py'>

### Specify fit parameters

In [66]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo'
        'vol' : 0, # max 10
        'exclude' : [], # put LECs here
        
        # semi-nnlo corrections
        'include_alpha_s' : True,
        'include_log' : True, 
        'include_log2' : True,
        'include_sunset' : False,
        
        # nnnlo corrections
        'include_latt_n3lo' : True,
    }, 
    'fit_type' : 'xpt-ratio', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; experimental: 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)
    'use_bijnens_central_value' : False,
    
    'use_prior' : True,
    'bias_correct' : True,
    'include_su2_isospin_corrrection' : False,
    'abbrs' : [u'a06m310L', u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_results' : False,
    'save_pickles' : False,
    'output_name' : 'fit_results',
}

In [67]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(fit_type=p_dict['fit_type'], order=p_dict['order']['fit'], F2=p_dict['F2'], 
                  include_log=p_dict['order']['include_log'], include_log2=p_dict['order']['include_log2'], 
                  include_sunset=p_dict['order']['include_sunset'], include_alpha_s=p_dict['order']['include_alpha_s'], 
                  include_latt_n3lo=p_dict['order']['include_latt_n3lo'], include_FV=(p_dict['order']['vol'] > 6), 
                  use_bijnens_central_value=p_dict['use_bijnens_central_value']
             )
    
# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, **p_dict)

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print(bootstrapper)
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info(),
                              output_name=p_dict['output_name'], save_pickles=p_dict['save_pickles'])

    
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

/mnt/Google/Documents/School Work/Research/fk-fpi/priors/xpt-ratio.csv
Using default prior.
Making fits...
Done!
100% complete
Time (s):  1.9708964824676514
Compiling results...

Model: xpt-ratio_FKFpi_nnlo_log_logSq_alphaS_a4

Fitting to nnlo 
 with volume corrections O(0) 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections: False):

F_K / F_pi = 1.1879(46) [1.1956(19)]   (delta_su2 = -0.00402(70))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
      A_a 0              -5.6 (2.8)            1            0.00000 (22)
          1               0.0 (5.0)        L_3 0            -0.0009 (28)
     A_aa 0                 48 (21)            1             0.0000 (31)
          1                  0 (50)        L_4 0           -0.00002 (34)
      A_k 0               3.9 (2.3)            1             0.0000 (50)
          1               0.0 (5.0)        L_5 0            

In [69]:
    # phys_params is a list: eg, ['mk'] or ['mk', 'mpi']
    def shift_fk_fpi_for_phys_params(self, phys_params, use_ratio=False):
        hbar_c = 197.32698045930 # MeV-fm
        lam2_chi = self.get_phys_point_data('lam2_chi')
        
        shifted_fkfpi = {}
        for abbr in self.abbrs:

            fkfpi_ens = self.plot_data[abbr]['FK'] / self.plot_data[abbr]['Fpi']
            fkfpi_fit_ens = self.extrapolate_to_ensemble(abbr)

            temp_data = {}
            for param in list(self.plot_data[abbr]):
                # Covert to phys units
                if param in ['mpi', 'mk']:
                    temp_data[param] = (self.plot_data[abbr][param] *hbar_c 
                                        / (self.plot_data[abbr]['a/w0'] *self.w0))
                else:
                    temp_data[param] = self.plot_data[abbr][param]
            
            for param in phys_params:
                temp_data[param] = self.get_phys_point_data(param)
                
            temp_data['lam2_chi'] = self.get_phys_point_data('lam2_chi')
            
            fkfpi_fit_phys = self.fk_fpi_fit_fcn(fit_data=temp_data)

            if use_ratio:
                shifted_fkfpi[abbr] = fkfpi_ens *(fkfpi_fit_phys / fkfpi_fit_ens)
            else:
                shifted_fkfpi[abbr] = fkfpi_ens + fkfpi_fit_phys - fkfpi_fit_ens

        return shifted_fkfpi
    

    def test(self):
        # used to convert to phys units
        hbar_c = 197.32698045930
        abbr = 'a09m220'
        data = self.plot_data
        
        return shift_fk_fpi_for_phys_params(self, phys_params=['mk'])
    
    test(bootstrapper)['a09m220']

array([1.1382(54), 1.1419(54), 1.1400(54), 1.1373(54), 1.1392(54),
       1.1382(54), 1.1400(54), 1.1378(54), 1.1369(54), 1.1367(54),
       1.1406(54), 1.1377(54), 1.1390(54), 1.1355(54), 1.1365(54),
       1.1382(54), 1.1390(54), 1.1385(54), 1.1403(54), 1.1388(54),
       1.1394(54), 1.1335(54), 1.1403(54), 1.1413(54), 1.1394(54),
       1.1419(54), 1.1368(54), 1.1380(54), 1.1391(54), 1.1399(54),
       1.1380(54), 1.1378(54), 1.1403(54), 1.1398(54), 1.1384(54),
       1.1404(54), 1.1390(54), 1.1374(54), 1.1400(54), 1.1416(54),
       1.1383(54), 1.1388(54), 1.1383(54), 1.1388(54), 1.1399(54),
       1.1383(54), 1.1395(54), 1.1384(54), 1.1362(54), 1.1396(54),
       1.1408(54), 1.1394(54), 1.1378(54), 1.1400(54), 1.1388(54),
       1.1376(54), 1.1386(54), 1.1421(54), 1.1378(54), 1.1392(54),
       1.1378(54), 1.1385(54), 1.1390(54), 1.1328(54), 1.1371(54),
       1.1376(54), 1.1413(54), 1.1409(54), 1.1368(54), 1.1390(54),
       1.1363(54), 1.1364(54), 1.1392(54), 1.1376(54), 1.1379(

In [61]:
    def fk_fpi_fit_fcn(self, fit_data=None, fit_parameters=None, fit_type=None, debug=None):
        if fit_data is None:
            fit_data = self.get_phys_point_data()
        if fit_parameters is None:
            fit_parameters = self.get_fit_parameters().copy()
        if fit_type is None:
            fit_type = self.fit_type


        if fit_type in ['ma', 'xpt']:
            fit_type = 'xpt'
        elif fit_type in ['ma-ratio', 'xpt-ratio']:
            fit_type = 'xpt-ratio'


        model = fit.fitter(order=self.order, fit_type=fit_type, F2=self.F2, fast_sunset=self.fast_sunset)._make_models()[0]
        return model.fitfcn(p=fit_parameters, fit_data=fit_data, debug=debug)
    
    fk_fpi_fit_fcn(bootstrapper)

1.1862(52)

In [60]:
bootstrapper.fk_fpi_fit_fcn()

1.1862(52)

In [26]:
list(bootstrapper.plot_data['a09m220'])

['FK',
 'Fpi',
 'mpi',
 'mk',
 'mss',
 'mju',
 'mjs',
 'mru',
 'mrs',
 'FK/Fpi',
 'a2DI',
 'a/w0',
 'L',
 'alpha_s',
 'lam2_chi']

In [24]:
list(bootstrapper.fit_data['a09m220'].keys())

['FK',
 'Fpi',
 'mpi',
 'mk',
 'mss',
 'mju',
 'mjs',
 'mru',
 'mrs',
 'FK/Fpi',
 'a2DI',
 'a/w0',
 'L',
 'alpha_s',
 'lam2_chi']

In [22]:
bootstrapper.get_fit_parameters()

{'A_a': -5.6(2.8),
 'A_aa': 48(21),
 'A_k': 3.9(2.3),
 'A_loga': -2.5(4.3),
 'A_p': 4.8(1.9),
 'L_1': -9(236)e-06,
 'L_2': -9(220)e-06,
 'L_3': -0.0009(28),
 'L_4': -0.00002(34),
 'L_5': 0.0002(10),
 'L_6': -2(100)e-06,
 'L_7': -0.00010(33),
 'L_8': -0.00001(14)}

In [20]:
np.array([1.1410(73), 1.1447(73), 1.1428(73), 1.1401(73), 1.1420(73),
       1.1411(73), 1.1428(73), 1.1406(73), 1.1398(73), 1.1396(73),
       1.1434(73), 1.1405(73), 1.1418(73), 1.1383(73), 1.1393(73),
       1.1410(73), 1.1418(73), 1.1413(73), 1.1431(73), 1.1416(73),
       1.1423(73), 1.1363(73), 1.1431(73), 1.1441(73), 1.1422(73),
       1.1447(73), 1.1396(73), 1.1408(73), 1.1419(73), 1.1427(73),
       1.1408(73), 1.1407(73), 1.1431(73), 1.1426(73), 1.1412(73),
       1.1432(73), 1.1418(73), 1.1402(73), 1.1428(73), 1.1444(73),
       1.1411(73), 1.1416(73), 1.1411(73), 1.1416(73), 1.1427(73),
       1.1411(73), 1.1423(73), 1.1413(73), 1.1390(73), 1.1424(73),
       1.1436(73), 1.1422(73), 1.1406(73), 1.1428(73), 1.1416(73),
       1.1404(73), 1.1414(73), 1.1449(73), 1.1406(73), 1.1420(73),
       1.1406(73), 1.1413(73), 1.1419(73), 1.1356(73), 1.1399(73),
       1.1404(73), 1.1441(73), 1.1437(73), 1.1396(73), 1.1418(73),
       1.1391(73), 1.1392(73), 1.1420(73), 1.1404(73), 1.1408(73),
       1.1413(73), 1.1400(73), 1.1409(73), 1.1413(73), 1.1417(73),
       1.1398(73), 1.1411(73), 1.1413(73), 1.1419(73), 1.1420(73),
       1.1418(73), 1.1394(73), 1.1421(73), 1.1431(73), 1.1411(73),
       1.1396(73), 1.1433(73), 1.1417(73), 1.1434(73), 1.1421(73),
       1.1411(73), 1.1427(73), 1.1415(73), 1.1394(73), 1.1395(73)],
      dtype=object)

<>:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:2: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:2: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:2: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:2: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:2: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:3: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:3: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:3: SyntaxWarning: 'float'

TypeError: 'float' object is not callable

In [34]:
fit_data = {
    'mpi' : 135.,
    'mk' : 495.5,
    'Fpi' : 92.2,
    'FK' : 110.5,
    'L' : np.inf, 
    'a/w0' : 0,
}
fit_data['lam2_chi'] = (4 *np.pi *fit_data['Fpi'])**2

fit_parameters = {
    'L_1' : 0.000372,
    'L_2' : 0.000493,
    'L_3' : -0.003070,
    'L_4' : 0.000089,
    'L_5' : 0.000377,
    'L_6' : 0.000011,
    'L_7' : -0.000340,
    'L_8' : 0.000294,
    'A_a' : 0.1,
    'A_k' : -3.000000,
    'A_p' : 4.000000
}


bootstrapper.fk_fpi_fit_fcn(fit_data=fit_data, fit_parameters=fit_parameters, debug=True)

NameError: name 'bootstrapper' is not defined